# 演習：Shorコードでの量子エラー訂正

```{contents} 目次
---
local: true
---
```

$\newcommand{\ket}[1]{|#1\rangle}$
$\newcommand{\braket}[2]{\langle #1 | #2 \rangle}$

量子プログラミング演習として、Toffoliゲートの基本ゲートでの分解を考え、それを使ってShorコードでエラー訂正をしてみましょう。

In [ ]:
# まずは必要なモジュールをインポートする
import sys
sys.path.append('/home/jovyan/qc-workbook-lecturenotes')
import numpy as np
from qiskit import QuantumCircuit, Aer, transpile
from qiskit.circuit import Parameter
from qiskit.quantum_info.operators import Operator
from IPython.display import Latex

from qc_workbook.show_state import show_state, statevector_expr

In [ ]:
def show_circuit_op(circuit, global_phase=0.):
    """Compiles the LaTeX expression of the operation of the circuit on computational basis states."""
    
    op = Operator(circuit)
    
    unitary = op.data * np.exp(-1.j * global_phase)
    
    ket_template = fr'|{{:0{circuit.num_qubits}b}}\rangle'

    exprs = list()
    for icol, col in enumerate(unitary.T):
        expr = statevector_expr(col, binary=True, state_label=None)
        exprs.append(fr'{ket_template.format(icol)} & \rightarrow {expr}')
        
    return Latex(r'\begin{align} ' + r' \\ '.join(exprs) + r' \end{align}')

## CPゲート

最初の問題は、制御$P$（$CP$）ゲートをCXと$R_z$から作ることです。おさらいをすると、CPゲートとは、2量子ビットゲートで、パラメータ$\phi$を取り、二つの量子ビットがともに$\ket{1}$であるような計算基底の位相を$\phi$前進させるゲートのことでした。

```{math}
    CP(\phi) \ket{00} & = \ket{00} \\
    CP(\phi) \ket{01} & = \ket{01} \\
    CP(\phi) \ket{10} & = \ket{10} \\
    CP(\phi) \ket{11} & = e^{i\phi} \ket{11}
```

**ヒント1**

まず制御$R_z$（$CR_z$）ゲートをCXと$R_z$から作ることを考えてみましょう。$CR_z$の作用は、左を制御ビット、右を標的ビットとして

```{math}
    CR_z(\phi) \ket{00} & = \ket{00} \\
    CR_z(\phi) \ket{01} & = \ket{01} \\
    CR_z(\phi) \ket{10} & = e^{-i/\phi/2} \ket{10} \\
    CR_z(\phi) \ket{11} & = e^{i\phi/2} \ket{11}
```

$CR_z$が作れれば、制御ビットに$R_z$をかけることで、$\ket{00}, \ket{01}, \ket{10}$の位相を揃えることができ、$CP$ゲートに全体位相がかかった状態が実現できます。全体位相は無視していいので、それで$CP$ゲートの完成です。

**ヒント2**

$R_z$を$X$で挟むと、$R_z$のパラメータの符号を反転させたのと同じ作用が得られます。

```{image} figs/rz_x_rz_x.png
:alt: rz_x_rz_x
:width: 400px
:align: center
```

In [ ]:
cp_circuit = QuantumCircuit(2, name='CP')

phi = Parameter('$\phi$')

# cp_circuit が CP(phi)を実装するようにゲートを加えてください。
# phiは普通の変数のように使えます。
# 例： cp_circuit.rz(phi, 1)

##################
### EDIT BELOW ###
##################

##################
### EDIT ABOVE ###
##################

cp_circuit.draw('mpl')

In [ ]:
# phiに具体的な値を入れて、CPが実装されていることを確認
phi_value = np.pi / 4.
show_circuit_op(cp_circuit.bind_parameters({phi: phi_value}), global_phase=(-phi_value / 4.))

$CP$は`QuantumCircuit`オブジェクトにもメソッド`cp`として備わっているので、以下では（回路図の見やすさを考慮して）標準メソッドを使うことにします。

## CCZゲート

次に、$CP$ゲートの特殊ケース（$\phi=\pi$）である制御$Z$（CZ）ゲートを、二重制御に拡張したCCZゲートを、$CP$とCXを組み合わせて実装します。

```{math}
    \text{CCZ} \ket{k} & = \ket{k} \; \text{for} 000 \leq k \leq 110 \\
    \text{CCZ} \ket{111} & = -\ket{111}
```

In [ ]:
ccz_circuit = QuantumCircuit(3, name='CCZ')

# ccz_circuit が CCZを実装するようにゲートを加えてください。

ccz_circuit.cp(np.pi / 2., 1, 0)
ccz_circuit.cp(np.pi / 2., 2, 0)

# 上の二行で|111>の符号が反転しますが、同時に|011>と|101>の符号も変わってしまうので、それを訂正する方法を考えてください。

##################
### EDIT BELOW ###
##################

##################
### EDIT ABOVE ###
##################

ccz_circuit.draw('mpl')

In [ ]:
show_circuit_op(ccz_circuit)

## Hadamardゲート

次にHadamardゲートの分解の$R_z$と$\sqrt{X}$への分解を考えます。$\sqrt{X}$の作用は

```{math}
    \sqrt{X} \ket{0} & = \frac{1}{\sqrt{2}} \left[e^{i\frac{\pi}{4}} \ket{0} + e^{-i\frac{\pi}{4}} \ket{1}\right] \\
    \sqrt{X} \ket{1} & = \frac{1}{\sqrt{2}} \left[e^{-i\frac{\pi}{4}} \ket{0} + e^{i\frac{\pi}{4}} \ket{1}\right]
```

です。

**ヒント**

$R_z(\phi)\ket{0}$や$R_z(\phi)\ket{1}$に$\sqrt{X}$をかけるとどうなるか、また$\sqrt{X}$をかけた後の状態に$R_z(\phi)$をかけたらどうなるか、計算してみましょう。今回も全体位相が生じます。

In [ ]:
h_circuit = QuantumCircuit(1, name='H')

# h_circuit が Hを実装するようにゲートを加えてください。

##################
### EDIT BELOW ###
##################

##################
### EDIT ABOVE ###
##################

h_circuit.draw('mpl')

In [ ]:
show_circuit_op(h_circuit, global_phase=(-1. / 4. * np.pi))

## Toffoliゲート

$Z$ゲートを$H$ゲートで挟むと$X$ゲートと等価になることを思い出して、CCZと$H$からCCXを作ってください。

In [ ]:
toffoli_circuit = QuantumCircuit(3, name='Toffoli')

# toffoli_circuit が Toffoliを実装するようにゲートを加えてください。
# 量子ビット0が標的、1と2が制御とします。
# Hadamardゲートには toffoli_circuit.h()を使って構いません。

##################
### EDIT BELOW ###
##################

##################
### EDIT ABOVE ###
##################

toffoli_circuit.draw('mpl')

In [ ]:
show_circuit_op(toffoli_circuit)

## 量子エラーとエラー訂正

冒頭で述べたように、量子エラーは有限個の量子演算子の作用として捉えることができます。特に、1量子ビットのエラーは、量子ビットに$X$、$Z$、$XZ$のどれかがかかることと等価です。$X$がかかるようなエラーをビット反転（bit flip）エラー、$Z$のケースを位相反転（phase flip）エラー、$XZ$のケースを複合（combined bit-phase flip）エラーと呼びます。

### ビット反転（$X$）エラー

Shorコードの元になるのはビット反転コードという、量子ビットを3つ使うエラー訂正コードです。ビット反転コードは、その名が示唆するようにビット反転エラーに対してのみ耐性を持ちます。論理量子ビットでの計算途中で3つの量子ビットのうちのどれかにビット反転エラーが一度だけ起きるとき、デコーディングの過程でエラーが補正されます。

ビット反転コードのエンコーディングは

```{math}
    \ket{0_L} & = \ket{000} \\
    \ket{1_L} & = \ket{111}
```

です。ここで$\ket{0_L}$と$\ket{1_L}$はそれぞれ「論理$\ket{0}$状態」と「論理$\ket{1}$状態」を表します。

以下では、適当な状態に初期化した量子ビットをビット反転コードでエンコードし、論理量子ビットに論理$Z$ゲートをかける過程でどれかひとつの（物理）量子ビットにビット反転エラーが起きるというシナリオを考えます。デコーディングの際にToffoliゲートを使うと、エラーが補正されます。

In [ ]:
bitflip_circuit = QuantumCircuit(3)

# データ量子ビット（第0ビット）を適当な状態に初期化
bitflip_circuit.u(0.2, 0.7, 0., 0)
bitflip_circuit.barrier()

# データビットの|0>が|000>に、|1>が|111>にエンコードされるように量子ビットをエンタングルさせてください
##################
### EDIT BELOW ###
##################

##################
### EDIT ABOVE ###
##################

bitflip_circuit.barrier()

# 論理ZゲートはZZZ
bitflip_circuit.z([0, 1, 2])

# 0から2のどれかひとつの量子ビットにX（ビット反転）がかかる
bitflip_circuit.x(np.random.randint(3))

bitflip_circuit.barrier()

# デコーディング：状態が|000>, |001>, |010>, |100>のときデータビットが|0>になり、
# |111>, |110>, |101>, |011>のとき|1>になるようにCXとToffoliを組んでください（Toffoliはすでに書き込んであります）
##################
### EDIT BELOW ###
##################

# 0が標的、1と2が制御
bitflip_circuit.ccx(1, 2, 0)

##################
### EDIT ABOVE ###
##################

bitflip_circuit.draw('mpl')

In [ ]:
show_state(bitflip_circuit, binary=True);

第1と第2量子ビット（Qiskitの順番なので左二桁）の状態が同一であれば正しくデコーディングされています。第0ビットの状態をエラーのない単一量子ビット回路のものと比較します。

In [ ]:
ref_circuit = QuantumCircuit(1)

ref_circuit.u(0.2, 0.7, 0., 0)
ref_circuit.z(0)
show_state(ref_circuit, binary=True);

### 位相反転（$Z$）エラー

次に、位相反転に対してのみ耐性を持つ3量子ビットのエラー訂正コードを考えます。$Z$を$H$で挟むと$X$になることを思い出すと、ビット反転コードを転用できることがすぐにわかります。

In [ ]:
phaseflip_circuit = QuantumCircuit(3)

phaseflip_circuit.u(0.2, 0.7, 0., 0)
phaseflip_circuit.barrier()

# エンコーディング
##################
### EDIT BELOW ###
##################

##################
### EDIT ABOVE ###
##################

phaseflip_circuit.barrier()

# 位相反転コードの論理ZゲートはXXX
phaseflip_circuit.x([0, 1, 2])

# ランダムに位相反転エラーが発生
phaseflip_circuit.z(np.random.randint(3))

phaseflip_circuit.barrier()

# デコーディング
##################
### EDIT BELOW ###
##################

phaseflip_circuit.ccx(1, 2, 0)

##################
### EDIT ABOVE ###
##################

phaseflip_circuit.draw('mpl')

In [ ]:
show_state(phaseflip_circuit, binary=True);

### Shorコード

Shorコードは位相反転コードの物理量子ビットにビット反転コードの論理量子ビットを使って作ります。全部で9つの量子ビットを使い、全ての1量子ビットエラー（$X, Z, XZ$）に耐性を持ちます。

In [ ]:
shor_circuit = QuantumCircuit(9)

shor_circuit.u(0.2, 0.7, 0., 0)
shor_circuit.barrier()

# エンコーディング
##################
### EDIT BELOW ###
##################

##################
### EDIT ABOVE ###
##################

shor_circuit.barrier()

# Shorコードの論理ZゲートもXXXXXXXXX
shor_circuit.x(range(9))

# ランダムにビットと位相が反転
erroneous_qubit = np.random.randint(9)
error = np.random.choice(['x', 'z', 'xz'])
if 'x' in error:
    shor_circuit.x(erroneous_qubit)
if 'z' in error:
    shor_circuit.z(erroneous_qubit)

shor_circuit.barrier()

# デコーディング
##################
### EDIT BELOW ###
##################

...

# ビット反転補正のためのToffoli
for itarg in [0, 3, 6]:
    shor_circuit.ccx(itarg + 1, itarg + 2, itarg)
    
...

# 位相反転補正のためのToffoli
shor_circuit.ccx(3, 6, 0)

##################
### EDIT ABOVE ###
##################

shor_circuit.draw('mpl')

In [ ]:
if error == 'xz':
    global_phase = -np.pi
else:
    global_phase = 0.

show_state(shor_circuit, global_phase=global_phase, binary=True);